<a href="https://colab.research.google.com/github/KaichengDING/Triple-Defense/blob/main/shuffle_net_idl_project_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install adversarial-robustness-toolbox

     |████████████████████████████████| 890kB 13.7MB/s 
     |████████████████████████████████| 235kB 32.4MB/s 
     |████████████████████████████████| 21.0MB 1.5MB/s 
     |████████████████████████████████| 747kB 47.4MB/s 


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np
import pandas as pd

import PIL

import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils import data

# from art.attacks.evasion import FastGradientMethod
# from art.attacks.evasion import ProjectedGradientDescentPyTorch
# from art.estimators.classification import PyTorchClassifier

import matplotlib.pyplot as plt
import time
import logging
import datetime
import random

cuda = torch.cuda.is_available()
cuda

True

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):      
      # CREATE A MASK FOR RANDOM PIXELS
      mask = np.random.uniform(low=0, high=1, size=tensor.size())
      mask[mask>=0.3] = 1
      mask[mask<0.3] = 0
      mask = torch.from_numpy(mask.astype(np.float32)).type(torch.FloatTensor)   
      # ADD GAUSSIAN NOISE AT RANDOM PIXELS
      res = tensor + mask * (torch.randn(tensor.size()) * self.std + self.mean) 
      return torch.clamp(input=res, min=0.0, max=1.0)
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
(np.random.uniform(low=0, high=1, size=(100,100)) < 0.05).sum()


510

In [ ]:
class ShuffleNet(nn.Module):
    def __init__(self, nb_classes =10):
        super(ShuffleNet, self).__init__()
        self.shuffle = models.shufflenet_v2_x2_0()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.shuffle(x)
        x = self.linear(x)
        return x


In [ ]:
class VGG16(nn.Module):
    def __init__(self, nb_classes=10):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16_bn()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.vgg16(x)
        x = self.linear(x)
        return x

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        with torch.cuda.amp.autocast():
          outputs = model(data)
          loss = criterion(outputs, target)
          running_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          total_predictions += target.size(0)
          correct_predictions += (predicted == target).sum().item()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        del data
        del target
    
    
    running_loss /= len(train_loader)
    acc = (correct_predictions/total_predictions)*100.0
    logging.info('Training Loss: {}'.format(running_loss))
    logging.info('Training Accuracy: {}%'.format(acc))
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            # outputs = model(data)

            # _, predicted = torch.max(outputs.data, 1)
            # total_predictions += target.size(0)
            # correct_predictions += (predicted == target).sum().item()

            # loss = criterion(outputs, target).detach()
            # running_loss += loss.item()

            with torch.cuda.amp.autocast():
              outputs = model(data)

              _, predicted = torch.max(outputs.data, 1)
              total_predictions += target.size(0)
              correct_predictions += (predicted == target).sum().item()

              loss = criterion(outputs, target).detach()
              running_loss += loss.item()
            

            del data
            del target

        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        logging.info('Testing Loss: {}'.format(running_loss))
        logging.info('Testing Accuracy: {}%'.format(acc))
        return running_loss, acc

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [ ]:
!pwd

/content


In [ ]:
# configure logging
logger = logging.getLogger("")

# reset handler
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

# set handler
stream_hdlr = logging.StreamHandler()
# file_hdlr = logging.FileHandler('/home/ubuntu/project/logs/'.format(datetime.datetime.now()))
file_hdlr = logging.FileHandler('/content/gdrive/MyDrive/IDL_Project/logs/log_{}.log'.format(datetime.datetime.now()))

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
stream_hdlr.setFormatter(formatter)
file_hdlr.setFormatter(formatter)

logger.addHandler(stream_hdlr)
logger.addHandler(file_hdlr)

logger.setLevel(logging.INFO)

In [ ]:
train_batchsize = 128
test_batchsize = 10
num_workers = 4
num_classes = 10

n_epochs = 20
img_size = 224
lr = 1e-3
min_lr = 1e-8
weight_decay = 5e-4
num_models = 10
noise_std = 0.1

device = torch.device("cuda" if cuda else "cpu")

hyper_params = {'lr': lr, 'min_lr': min_lr, 'weight_decay': weight_decay, 'num_models': num_models, 'num_epochs': n_epochs, 'noise_std': noise_std}
logging.info(hyper_params)


2020-12-11 01:03:17,797 INFO {'lr': 0.001, 'min_lr': 1e-08, 'weight_decay': 0.0005, 'num_models': 10, 'num_epochs': 20, 'noise_std': 0.1}


In [ ]:
seeds = np.arange(11, 20)

for seed in seeds:
#   logging.info('Seed: {}'.format(seed))
  torch.manual_seed(seed)
  

  train_transform = transforms.Compose([transforms.Resize(size=img_size),
                      transforms.ColorJitter(hue=.05, saturation=.05),
                      transforms.RandomHorizontalFlip(p=0.5),
                      transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
                      transforms.ToTensor(),
                      transforms.Normalize((0, 0, 0), (1, 1, 1)),
                      AddGaussianNoise(0., noise_std)])

  test_transform = transforms.Compose([transforms.Resize(size=img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0, 0, 0), (1, 1, 1)),
                     AddGaussianNoise(0., noise_std)])

  
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batchsize, shuffle=True, num_workers=num_workers)
  testloader = torch.utils.data.DataLoader(testset, batch_size=test_batchsize, shuffle=False, num_workers=num_workers)

  model = ShuffleNet(num_classes)
  model.apply(init_weights)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, cooldown=5, min_lr=min_lr, verbose=True)
  scaler = torch.cuda.amp.GradScaler()
  model.to(device)

  # Train_loss = []
  # Test_loss = []
  # Test_acc = []

  for i in range(n_epochs):
    tic = time.time()
    # logging.info('Epoch: {}'.format(i))
    train_loss = train_epoch(model, trainloader, criterion, optimizer)
    test_loss, test_acc = test_model(model, testloader, criterion)
    scheduler.step(test_loss)
    # Train_loss.append(train_loss)
    # Test_loss.append(test_loss)
    # Test_acc.append(test_acc)
    toc = time.time()
    print(toc-tic)
    # logging.info('Time: {}s'.format(toc - tic))
    # logging.info('='*20)
    
  torch.save({'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()}, 
        "/content/gdrive/My Drive/IDL_Project/models_dropout/Model_shuffle_dropout{}_{}".format(0.3,datetime.datetime.now()))
  
  torch.cuda.empty_cache()
  del model
  del criterion
  del optimizer
  del scheduler
  del scaler
  del trainloader
  del testloader
  del train_transform
  del test_transform

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


2020-12-11 01:08:50,810 INFO Training Loss: 1.9384690962179238
2020-12-11 01:08:50,812 INFO Training Accuracy: 36.65%
2020-12-11 01:09:29,674 INFO Testing Loss: 1.295907773822546
2020-12-11 01:09:29,675 INFO Testing Accuracy: 52.019999999999996%


351.79953813552856


2020-12-11 01:14:41,823 INFO Training Loss: 1.1709146722198447
2020-12-11 01:14:41,825 INFO Training Accuracy: 57.962%
2020-12-11 01:15:20,051 INFO Testing Loss: 1.01713315859437
2020-12-11 01:15:20,052 INFO Testing Accuracy: 63.339999999999996%


350.3768274784088


...

347.31152510643005


2020-12-11 18:29:11,939 INFO Training Loss: 0.2408757230357441
2020-12-11 18:29:11,941 INFO Training Accuracy: 91.77199999999999%
2020-12-11 18:29:49,851 INFO Testing Loss: 0.28880053196504013
2020-12-11 18:29:49,852 INFO Testing Accuracy: 90.41%


349.15129685401917


2020-12-11 18:35:00,250 INFO Training Loss: 0.2224740916315247
2020-12-11 18:35:00,252 INFO Training Accuracy: 92.5%
2020-12-11 18:35:37,796 INFO Testing Loss: 0.2876993918819353
2020-12-11 18:35:37,798 INFO Testing Accuracy: 90.14%


347.9455280303955


# Get test examples

In [ ]:
test_transform = transforms.Compose([transforms.Resize(size=img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0, 0, 0), (1, 1, 1))])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batchsize, shuffle=False, num_workers=num_workers)

# x_test = []
# y_test = []
# for batch_idx, (X, Y) in enumerate(testloader):
#   x_test.append(X.numpy())
#   y_test.append(Y.numpy())
# x_test = np.concatenate(x_test)
# y_test = np.concatenate(y_test)

Files already downloaded and verified


# Ensemble Parameters

# Load Saved Models

In [ ]:
model_names = ['Model_45_0.1', 
               'Model_46_0.1', 
               'Model_47_0.1', 
               'Model_48_0.1',
               'Model_49_0.1',
               'Model_50_0.1']
# model_names = ['Model_45_0.1', 
#                'Model_46_0.1', 
#                'Model_47_0.1']
num_models = len(model_names)
num_models_selected = 4
num_classes = 10
print(num_models)

6


In [ ]:
model_dict = {}
for model_name in model_names:
  model = VGG16()
  model_data = torch.load('/content/gdrive/My Drive/IDL_Project/modelS/{}'.format(model_name), map_location=torch.device('cpu'))
  model.load_state_dict(model_data['model_state_dict'])
  model_dict[model_name] = model

In [ ]:
!nvidia-smi
# model_list = []
# for model_name in model_names:
#   model = VGG16()
#   model_data = torch.load('/content/gdrive/My Drive/IDL_Project/models/{}'.format(model_name), map_location=torch.device('cpu'))
#   model.load_state_dict(model_data['model_state_dict'])
#   model_list.append(model)
# !nvidia-smi

Mon Nov  9 21:44:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    34W / 250W |   7577MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Ensemble Def1

In [ ]:
class MyEnsemble(nn.Module):
    def __init__(self, model_dict, num_models_selected, num_classes=10):
        super(MyEnsemble, self).__init__()
        self.model_dict = model_dict
        self.num_models = len(model_dict)
        self.num_models_selected = num_models_selected
        self.num_classes = num_classes
        
    def forward(self, x):
        indices = [i for i in range(self.num_models)]
        random.shuffle(indices)
        indices = indices[:self.num_models_selected]
        models = [*self.model_dict]
        models_selected = [models[idx] for idx in indices]

        logits_list = []
        labels_list = []

        noise = torch.randn(x.size()) * noise_std
        noise = noise.to(device)
        x_noised = x + noise

        for model_name in models_selected:
          # model = model.to(device)
          model = self.model_dict[model_name]
          logits = model(x_noised)
          label_out = torch.unsqueeze(torch.argmax(torch.nn.functional.softmax(logits, dim=0), dim=1), dim=0)
          logits = torch.unsqueeze(logits, dim=0)
          labels_list.append(label_out)
          logits_list.append(logits)
          # model = model.to('cpu')
          # print("Model on CUDA: {}".format(next(model.parameters()).is_cuda))
        
        # majority vote
        labels_tensor = torch.cat(labels_list, dim=0)
        logits_tensor = torch.cat(logits_list, dim=0)
        
        del noise
        del logits_list
        del labels_list

        voted_class = torch.max(labels_tensor, dim=0) # possibly ties
        mask = (labels_tensor == voted_class.values).int()
        mask = mask.unsqueeze(dim=1).repeat(1, num_classes, 1)
        mask = mask.transpose(1, 2)

        sum_logits = torch.sum(mask * logits_tensor, dim=0)
        divider = torch.sum(mask, dim=0)
        
        output = sum_logits / divider


        del labels_tensor
        del logits_tensor
        del voted_class
        del mask
        del sum_logits
        del divider
        models_selected = []
        
        torch.cuda.empty_cache()

        return output

In [ ]:
# class MyEnsemble(nn.Module):
#     def __init__(self, model_list, num_models_selected, num_classes=10):
#         super(MyEnsemble, self).__init__()
#         self.model_list = []
#         self.num_models = len(model_list)
#         self.num_models_selected = num_models_selected
#         self.num_classes = num_classes
#         for model in model_list:
#           self.model_list.append(model)
        
#     def forward(self, x):
#         indices = [i for i in range(self.num_models)]
#         random.shuffle(indices)
#         indices = indices[:self.num_models_selected]
#         models_selected = [self.model_list[idx] for idx in indices]

#         logits_list = []
#         labels_list = []

#         noise = torch.randn(x.size()) * noise_std
#         noise = noise.to(device)
#         x_noised = x + noise

#         for model in models_selected:
#           # model = model.to(device)
#           logits = model(x_noised)
#           label_out = torch.unsqueeze(torch.argmax(torch.nn.functional.softmax(logits, dim=0), dim=1), dim=0)
#           logits = torch.unsqueeze(logits, dim=0)
#           labels_list.append(label_out)
#           logits_list.append(logits)
#           # model = model.to('cpu')
#           # print("Model on CUDA: {}".format(next(model.parameters()).is_cuda))
        
#         # majority vote
#         labels_tensor = torch.cat(labels_list, dim=0)
#         logits_tensor = torch.cat(logits_list, dim=0)
        
#         del noise
#         del logits_list
#         del labels_list

#         voted_class = torch.max(labels_tensor, dim=0) # possibly ties
#         mask = (labels_tensor == voted_class.values).int()
#         mask = mask.unsqueeze(dim=1).repeat(1, num_classes, 1)
#         mask = mask.transpose(1, 2)

#         sum_logits = torch.sum(mask * logits_tensor, dim=0)
#         divider = torch.sum(mask, dim=0)
        
#         output = sum_logits / divider


#         del labels_tensor
#         del logits_tensor
#         del voted_class
#         del mask
#         del sum_logits
#         del divider
#         models_selected = []
        
#         torch.cuda.empty_cache()

#         return output
    

#     # def get_logits(self, x):
#     #     # generate random indices
#     #     indices = [i for i in range(self.num_models)]
#     #     random.shuffle(indices)
#     #     indices = indices[:self.num_models_selected]
#     #     models_selected = [self.model_list[idx] for idx in indices]

#     #     noise = torch.randn(x.size()) * noise_std
#     #     noise = noise.to(device)
#     #     x_noised = x + noise

#     #     logits_list = []
        
#     #     for model in models_selected:
#     #         model = model.to(device)
#     #         logits = model(x_noised)
#     #         logits_list.append(logits)
#     #         model = model.to('cpu')
        
#     #     return logits_list

    
#     def predict(self, x):
#         raise NotImplemented("predict")


# class MyPyTorchClassifier(PyTorchClassifier):
#     def loss_gradient(self, x: np.ndarray, y: np.ndarray, **kwargs) -> np.ndarray:
#         """
#         Compute the gradient of the loss function w.r.t. `x`.
#         :param x: Sample input with shape as expected by the model.
#         :param y: Target values (class labels) one-hot-encoded of shape `(nb_samples, nb_classes)` or indices of shape
#                   `(nb_samples,)`.
#         :return: Array of gradients of the same shape as `x`.
#         """
#         import torch  # lgtm [py/repeated-import]

#         # Apply preprocessing
#         x_preprocessed, y_preprocessed = self._apply_preprocessing(x, y, fit=False)

#         # Check label shape
#         y_preprocessed = self.reduce_labels(y_preprocessed)

#         # Convert the inputs to Tensors
#         inputs_t = torch.from_numpy(x_preprocessed).to(self._device)
#         inputs_t.requires_grad = True

#         # Convert the labels to Tensors
#         labels_t = torch.from_numpy(y_preprocessed).to(self._device)

#         #######################################
#         # Compute the gradient and return

#         logits_list, _ = self._model(inputs_t)

#         loss = torch.Tensor(0.0)
#         # loss = loss.to(device)

#         for logits in logits_list:
#           loss += self._loss(logits, labels_t)
          
#           # Clean gradients
#           self._model.zero_grad()

#         # Compute gradients
#         loss.backward()
#         grads = inputs_t.grad.cpu().numpy().copy()  # type: ignore
#         grads = self._apply_preprocessing_gradient(x, grads)
#         assert grads.shape == x.shape

#         return grads

      
#     def loss_gradient_framework(self, x: np.ndarray, y: np.ndarray, **kwargs) -> np.ndarray:
#         """
#         Compute the gradient of the loss function w.r.t. `x`.
#         :param x: Sample input with shape as expected by the model.
#         :param y: Target values (class labels) one-hot-encoded of shape `(nb_samples, nb_classes)` or indices of shape
#                   `(nb_samples,)`.
#         :return: Array of gradients of the same shape as `x`.
#         """
#         import torch  # lgtm [py/repeated-import]

#         # Apply preprocessing
#         x_preprocessed, y_preprocessed = self._apply_preprocessing(x, y, fit=False)

#         # Check label shape
#         y_preprocessed = self.reduce_labels(y_preprocessed)

#         # Convert the inputs to Tensors
#         inputs_t = torch.from_numpy(x_preprocessed).to(self._device)
#         inputs_t.requires_grad = True

#         # Convert the labels to Tensors
#         labels_t = torch.from_numpy(y_preprocessed).to(self._device)

#         #######################################
#         # Compute the gradient and return
#         logits_list = self._model.get_logits(inputs_t)

#         # loss = torch.Tensor(0.0)
#         # loss = loss.to(device)

#         loss = self._loss(logits[0], labels_t)

#         for i in range(1, len(logits_list)):
#           loss += self._loss(logits_list[i], labels_t)
          
#           # Clean gradients
#           self._model.zero_grad()

#         # Compute gradients
#         loss.backward()
#         grads = inputs_t.grad.cpu().numpy().copy()  # type: ignore
#         grads = self._apply_preprocessing_gradient(x, grads)
#         assert grads.shape == x.shape

#         return grads

In [ ]:
# class MyEnsemble(nn.Module):
#     def __init__(self, model_list, num_models_selected, num_classes=10):
#         super(MyEnsemble, self).__init__()
#         self.model_list = []
#         self.num_models = len(model_list)
#         self.num_models_selected = num_models_selected
#         self.num_classes = num_classes
#         for model in model_list:
#           self.model_list.append(model)
        
#     def forward(self, x):
#         indices = [i for i in range(self.num_models)]
#         random.shuffle(indices)
#         indices = indices[:self.num_models_selected]
#         models_selected = [self.model_list[idx] for idx in indices]

#         logits_list = []
#         labels_list = []

#         noise = torch.randn(x.size()) * noise_std
#         noise = noise.to(device)
#         x_noised = x + noise

#         for model in models_selected:
#           model = model.to(device)
#           logits = model(x_noised)
#           label_out = torch.unsqueeze(torch.argmax(torch.nn.functional.softmax(logits, dim=0), dim=1), dim=0)
#           logits = torch.unsqueeze(logits, dim=0)
#           labels_list.append(label_out)
#           logits_list.append(logits)
#           model = model.to('cpu')
        
#         # majority vote
#         labels_tensor = torch.cat(labels_list, dim=0)
#         logits_tensor = torch.cat(logits_list, dim=0)
#         voted_class = torch.max(labels_tensor, dim=0) # possibly ties
#         mask = (labels_tensor == voted_class.values).int()

#         mask = mask.unsqueeze(dim=1).repeat(1, num_classes, 1)
#         mask = mask.transpose(1, 2)

#         sum_logits = torch.sum(mask * logits_tensor, dim=0)
#         divider = torch.sum(mask, dim=0)
        
#         output = sum_logits / divider

#         del logits_list
#         del labels_list
#         del labels_tensor
#         del logits_tensor
#         del voted_class
#         del mask
#         del sum_logits
#         del divider
#         models_selected = []
#         return output

#         # max_logits = torch.max(mask * logits_tensor, dim=0)
#         # return max_logits.values


# Ensemble Def2

In [ ]:
# class MyEnsemble(nn.Module):
#     def __init__(self, model_list, num_models_selected, num_classes=10):
#         super(MyEnsemble, self).__init__()
#         self.model_list = []
#         self.num_models = len(model_list)
#         self.num_models_selected = num_models_selected
#         self.num_classes = num_classes
#         for model in model_list:
#           self.model_list.append(model)
#         # Remove last linear layer
#         self.softmax = nn.Softmax()
        
#     def forward(self, x):
#         indices = [i for i in range(self.num_models)]
#         random.shuffle(indices)
#         indices = indices[:self.num_models_selected]
#         model_names_selected = [self.model_list[idx] for idx in indices]
        
#         models_selected = []
        
#         for model_name in model_names_selected:
#           model = VGG16()
#           model_data = torch.load('./{}'.format(model_name))
#           model.load_state_dict(model_data['model_state_dict'])
#           model.to(device)
#           models_selected.append(model)

#         logits_list = []
#         labels_list = []

#         noise = torch.randn(x.size()) * noise_std
#         noise = noise.to(device)
#         x_noised = x + noise

#         for model in models_selected:
#           logits = model(x_noised)
#           label_out = torch.unsqueeze(torch.argmax(self.softmax(logits), dim=1), dim=0)
#           logits = torch.unsqueeze(logits, dim=0)
#           labels_list.append(label_out)
#           logits_list.append(logits)
        
#         # majority vote
#         labels_tensor = torch.cat(labels_list, dim=0)
#         logits_tensor = torch.cat(logits_list, dim=0)
#         voted_class = torch.max(labels_tensor, dim=0) # possibly ties
#         mask = (labels_tensor == voted_class.values).int()

#         mask = mask.unsqueeze(dim=1).repeat(1, num_classes, 1)
#         mask = mask.transpose(1, 2)

#         sum_logits = torch.sum(mask * logits_tensor, dim=0)
#         divider = torch.sum(mask, dim=0)
        
#         output = sum_logits / divider

#         for model in models_selected:
#             del model
#         del models_selected
#         del labels_list
#         del logits_list
#         del labels_tensor
#         del logits_tensor
#         del voted_class
#         del mask
#         del sum_logits
#         del divider

#         return output

#         # max_logits = torch.max(mask * logits_tensor, dim=0)
#         # return max_logits.values

# Ensemble Class Definition

# Create classifier object

In [ ]:
# for model in model_list:
#   model = model.to(device)
for key in model_dict:
  model_dict[key] = model_dict[key].to(device)

In [ ]:
!nvidia-smi

Mon Nov  9 17:17:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    37W / 250W |   4031MiB / 16280MiB |     26%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
my_ensemble = MyEnsemble(model_dict, num_models_selected, num_classes=10)
# my_ensemble = MyEnsemble(model_names, num_models_selected, num_classes=10)
criterion = nn.CrossEntropyLoss()
classifier = PyTorchClassifier(
    model=my_ensemble,
    clip_values=(0, 1),
    loss=criterion,
    input_shape=(3, 224, 224),
    nb_classes=10,
)
# attack = FastGradientMethod(estimator=classifier, eps=0.01)
# attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps=0.1, eps_step=0.01)

2020-11-09 14:11:20,544 INFO Inferred 1 hidden layers on PyTorch classifier.


In [ ]:
pgd_epsilon = 8.0 / 255.0 / np.array([2.0, 4.0, 8.0, 16.0])
diff = np.array([1e-3, 1e-4, 1e-5, 1e-6])
pgd_epsilon_step = pgd_epsilon - diff
fgm_epsilon = 0.01
num_selected = [2,3,6]
num_selected

[2, 3, 6]

In [ ]:
for i, models_selected in enumerate(num_selected):
    my_ensemble = MyEnsemble(model_dict, models_selected, num_classes=10)
# my_ensemble = MyEnsemble(model_names, num_models_selected, num_classes=10)
    criterion = nn.CrossEntropyLoss()
    classifier = PyTorchClassifier(
                     model=my_ensemble,
                     clip_values=(0, 1),
                     loss=criterion,
                     input_shape=(3, 224, 224),
                     nb_classes=10,
                     )
    attack = FastGradientMethod(estimator=classifier, eps=0.01)
    Acc = []
    for batch_idx, (X, Y) in enumerate(testloader):
        x_test = X.numpy()
        y_test = Y.numpy()
        x_test_adv = attack.generate(torch.from_numpy(x_test))
        predictions = classifier.predict(x_test_adv)
        accuracy = np.sum(np.argmax(predictions, axis=1) == np.array(y_test)) / len(y_test)
        Acc.append(accuracy)
        logging.info('accuracy: {}'.format(accuracy))
  
    logging.info("Accuracy for {} selected models {}".format( models_selected, np.mean(np.array(Acc))))

    pred_list = np.asanyarray(Acc)
    idx = np.arange(0,len(testset)/test_batchsize).tolist()
    dict = {'Id': idx ,'acc': pred_list} 
    df = pd.DataFrame(dict) 
    # saving the dataframe 
    df.to_csv('results/fgm_num_selected_result_eps{}_{}.csv'.format(models_selected, datetime.datetime.now()),index=False)


Streaming output truncated to the last 5000 lines.
2020-11-09 22:02:03,391 INFO accuracy: 0.7
2020-11-09 22:02:03,398 INFO Using model predictions as correct labels for FGM.
2020-11-09 22:02:04,280 INFO Success rate of FGM attack: 30.00%
2020-11-09 22:02:04,423 INFO accuracy: 0.6
2020-11-09 22:02:04,430 INFO Using model predictions as correct labels for FGM.
2020-11-09 22:02:05,326 INFO Success rate of FGM attack: 30.00%
2020-11-09 22:02:05,467 INFO accuracy: 0.7
2020-11-09 22:02:05,475 INFO Using model predictions as correct labels for FGM.
2020-11-09 22:02:06,359 INFO Success rate of FGM attack: 0.00%
2020-11-09 22:02:06,499 INFO accuracy: 1.0
2020-11-09 22:02:06,506 INFO Using model predictions as correct labels for FGM.
2020-11-09 22:02:07,388 INFO Success rate of FGM attack: 30.00%
2020-11-09 22:02:07,527 INFO accuracy: 0.7
2020-11-09 22:02:07,534 INFO Using model predictions as correct labels for FGM.
2020-11-09 22:02:08,425 INFO Success rate of FGM attack: 50.00%
2020-11-09 22:0

In [ ]:
# attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps=0.03, eps_step=0.02)

In [ ]:
!ls results/

# Generating attack samples

In [ ]:
Acc = []
for batch_idx, (X, Y) in enumerate(testloader):
  x_test = X.numpy()
  y_test = Y.numpy()
  # x_test = x_test[:10]
  # y_test = y_test[:10]
  x_test_adv = attack.generate(torch.from_numpy(x_test))
  predictions = classifier.predict(x_test_adv)
  accuracy = np.sum(np.argmax(predictions, axis=1) == np.array(y_test)) / len(y_test)
  Acc.append(accuracy)
  logging.info('accuracy: {}'.format(accuracy))



Streaming output truncated to the last 5000 lines.
PGD - Random Initializations: 100%|██████████| 1/1 [00:25<00:00, 25.04s/it]
2020-11-09 02:55:26,931 INFO Success rate of attack: 90.00%
2020-11-09 02:55:27,033 INFO accuracy: 0.1
PGD - Random Initializations: 100%|██████████| 1/1 [00:25<00:00, 25.05s/it]
2020-11-09 02:55:52,476 INFO Success rate of attack: 60.00%
2020-11-09 02:55:52,578 INFO accuracy: 0.3
PGD - Random Initializations: 100%|██████████| 1/1 [00:24<00:00, 24.99s/it]
2020-11-09 02:56:17,962 INFO Success rate of attack: 100.00%
2020-11-09 02:56:18,064 INFO accuracy: 0.2
PGD - Random Initializations: 100%|██████████| 1/1 [00:25<00:00, 25.01s/it]
2020-11-09 02:56:43,469 INFO Success rate of attack: 90.00%
2020-11-09 02:56:43,571 INFO accuracy: 0.2
PGD - Random Initializations: 100%|██████████| 1/1 [00:25<00:00, 25.03s/it]
2020-11-09 02:57:08,999 INFO Success rate of attack: 70.00%
2020-11-09 02:57:09,100 INFO accuracy: 0.4
PGD - Random Initializations: 100%|██████████| 1/1 [0

In [ ]:
import pandas as pd

pred_list = np.asanyarray(Acc)
idx = np.arange(0,len(testset)/test_batchsize).tolist()
dict = {'Id': idx ,'acc': pred_list} 
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv('./gdrive/My Drive/IDL_Project/results/FGM_result_{}.csv'.format(datetime.datetime.now()),index=False)

In [ ]:
np.mean(np.array(Acc))

0.7423

In [ ]:
x_test_adv = attack.generate(torch.from_numpy(x_test[:1000]))

2020-11-08 22:37:28,336 INFO Using model predictions as correct labels for FGM.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
2020-11-08 22:52:35,795 INFO Success rate of FGM attack: 29.50%


In [ ]:
!nvidia-smi

Sun Nov  8 22:34:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    36W / 300W |  12647MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.array(y_test)[:100]) / 100
print(accuracy)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
